# Experiment with intermediate answers

In [1]:
from prompting.prompts import INTERMEDIATE_ANSWER_PROMPT_TEMPLATE

In [2]:
question = "Andy's car fuel efficiency is 10 MPG (miles per gallon). If the current price for regular gas is $3/gallon, how much money is Andy's car consuming per week if he only uses his car to go to work from Monday to Friday and the one-way distance between his home and office is 5 miles?"
cot_steps = [
      "Calculate the total distance Andy drives per day: 5 miles (one way) * 2 = 10 miles.",
      "Calculate the total distance Andy drives per week: 10 miles/day * 5 days = 50 miles.",
      "Calculate the number of gallons Andy consumes per week: 50 miles / 10 MPG = 5 gallons.",
      "Calculate the total cost of gas per week: 5 gallons * $3/gallon = $15.",
      "Therefore, Andy's car consumes $15 per week."
    ]

In [3]:
cot_steps_2 = cot_steps[:2]
cot_steps_3 = cot_steps[:3]

In [4]:
filled_prompt_2 = INTERMEDIATE_ANSWER_PROMPT_TEMPLATE.format(
    question=question,
    cot_steps='"' + '",\n\t\t"'.join(cot_steps_2) + '"'
)

In [5]:
print(filled_prompt_2)

Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calculate t

In [6]:
filled_prompt_3 = INTERMEDIATE_ANSWER_PROMPT_TEMPLATE.format(
    question=question,
    cot_steps='"' + '",\n\t\t"'.join(cot_steps_3) + '"'
)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList

model_name = "google/gemma-3-1b-it"
tokenizer_generate = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model_generate = AutoModelForCausalLM.from_pretrained(model_name)

C:\Users\libon\anaconda3\envs\bridging_chains\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [30]:
from transformers import StoppingCriteria, StoppingCriteriaList
class StopAfterIntermediateAnswer(StoppingCriteria):
    def __init__(self, tokenizer, stop_strings = ("\n", "\n\n")):
        super().__init__()
        self.stop_strings = [tokenizer.encode(stop_string, add_special_tokens=False)[0] for stop_string in stop_strings]

    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0, -1] in self.stop_strings

In [31]:
stop_criteria = StoppingCriteriaList([StopAfterIntermediateAnswer(tokenizer_generate)])

In [32]:
tokenized_2 = tokenizer_generate(filled_prompt_2, return_tensors="pt")
output_tokenized = model_generate.generate(**tokenized_2, max_new_tokens=200, do_sample=True, stopping_criteria=stop_criteria)
output = tokenizer_generate.decode(output_tokenized[0])
print(output)

<bos>Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calcul

In [33]:
tokenized_3 = tokenizer_generate(filled_prompt_3, return_tensors="pt")
output_tokenized_3 = model_generate.generate(**tokenized_3, max_new_tokens=200, do_sample=True, stopping_criteria=stop_criteria)
output3 = tokenizer_generate.decode(output_tokenized_3[0])
print(output3)

<bos>Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calcul

In [36]:
# Extract final answer
splitted_output = output.split('"final_answer": ')[-1].replace('\n', '').replace('"', '')
splitted_output

'The answer is $15.00'

## Entailment

In [37]:
answer1 = "Andy’s car consumes $15 per week."
answer2 = "Andy’s car consumes $17 per week."

In [38]:
from transformers import AutoModelForSequenceClassification

nli_model_name = "cross-encoder/nli-deberta-v3-large"
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)
nli_model.eval()

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

C:\Users\libon\anaconda3\envs\bridging_chains\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\libon\.cache\huggingface\hub\models--cross-encoder--nli-deberta-v3-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

In [54]:
# Bidirectional Entailment
# Direction 1
premise = "Question: " + question + " " + "Answer: " + answer1
hypothesis = "Question: " + question + " " + "Answer: " + answer2
print(premise + " => " + hypothesis)
inp = nli_tokenizer(premise, hypothesis, return_tensors="pt", padding=True)
outp = nli_model(**inp)
outp

Question: Andy's car fuel efficiency is 10 MPG (miles per gallon). If the current price for regular gas is $3/gallon, how much money is Andy's car consuming per week if he only uses his car to go to work from Monday to Friday and the one-way distance between his home and office is 5 miles? Answer: Andy’s car consumes $15 per week. => Question: Andy's car fuel efficiency is 10 MPG (miles per gallon). If the current price for regular gas is $3/gallon, how much money is Andy's car consuming per week if he only uses his car to go to work from Monday to Friday and the one-way distance between his home and office is 5 miles? Answer: Andy’s car consumes $17 per week.


SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5661, -3.2943,  0.4860]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [55]:
import torch
predictions = torch.softmax(outp["logits"], dim=-1)
print(predictions)

tensor([[0.7422, 0.0057, 0.2520]], grad_fn=<SoftmaxBackward0>)


In [58]:
# Direction 2
inp = nli_tokenizer(hypothesis, premise, return_tensors="pt", padding=True)
outp = nli_model(**inp)
predictions = torch.softmax(outp.logits, dim=-1)
print(predictions)

tensor([[0.6945, 0.0138, 0.2916]], grad_fn=<SoftmaxBackward0>)


In [59]:
predictions[0].argmax().item()

0